In [142]:
%matplotlib inline

import sys
import traceback
from IPython.display import display

import numpy as np
import pandas as pd
import tensorflow as tf
from matplotlib import pyplot as plt

from sklearn import decomposition as decomp
from sklearn import ensemble as ens
from sklearn import linear_model as lm
from sklearn import metrics
from sklearn import model_selection as ms
from sklearn import neighbors as neigh
from sklearn import neural_network as nn
from sklearn import preprocessing as pre
from sklearn import svm

In [124]:
'''Preprocessing Initialisation'''
std_scaler = pre.StandardScaler()
unit_scaler = pre.MinMaxScaler()

In [164]:
'''Data loading and initial manipulation'''
df = pd.read_csv('C:\\Users\mjs3607\Documents\Stapleton\'s Lab\Spectral Data\headers3mgperml.csv')
x = df.iloc[:, 15:]
y_orig = df.iloc[:, 1:14]
# ohe.fit(y_orig)
# print("{}\n{}".format(ohe.n_values_, ohe.feature_indices_))

y_cat, y_num = pd.DataFrame(df.iloc[:, 1:5]), df.iloc[:, 5:14].drop('PKID', 1, inplace=False)
y = pd.concat((y_cat, y_num), axis=1)

headers = list(y_orig)
cat_headers = list(y_orig.iloc[:, 0:4])
num_headers = list(y_orig.iloc[:, 4:14].drop('PKID', 1, inplace=False))
# print("{}\n{}\n{}".format(headers, cat_headers, num_headers))
# print(y)

In [165]:
'''Creating training and testing sets'''
x_train, x_test, y_train, y_test = ms.train_test_split(x, y, random_state=1)
xc_train, xc_test, yc_train, yc_test = ms.train_test_split(x, y_cat, random_state=1)
xn_train, xn_test, yn_train, yn_test = ms.train_test_split(x, y_num, random_state=1)

col_train = {}
col_test = {}

for i in range(len(headers)):
    xtmp_train, xtmp_test, ytmp_train, ytmp_test = ms.train_test_split(x, y_orig.iloc[:, i], random_state=1)
    col_train[headers[i]] = (xtmp_train, ytmp_train)
    col_test[headers[i]] = (xtmp_test, ytmp_test)

print("x_train: {}\tx_test: {}\ty_train: {}\ty_test: {}".format(x_train.shape, x_test.shape, y_train.shape, y_test.shape))
print("xc_train: {}\txc_test: {}\tyc_train: {}\tyc_test: {}".format(xc_train.shape, xc_test.shape, yc_train.shape, yc_test.shape))
print("xn_train: {}\txn_test: {}\tyn_train: {}\tyn_test: {}".format(xn_train.shape, xn_test.shape, yn_train.shape, yn_test.shape))
# print("y_orig_train: {}\t\ty_orig_test: {}\n\n".format(y_orig_train.shape, y_orig_test.shape))

# print(y_train)
# print("\n\n")
# print(y_test)

x_train: (34, 240)	x_test: (12, 240)	y_train: (34, 12)	y_test: (12, 12)
xc_train: (34, 240)	xc_test: (12, 240)	yc_train: (34, 4)	yc_test: (12, 4)
xn_train: (34, 240)	xn_test: (12, 240)	yn_train: (34, 8)	yn_test: (12, 8)


In [166]:
y_num_std = (y_num - y_num.mean())/y_num.std()
y_std = pd.concat((y_cat, y_num_std), axis=1)

std_col_train = {}
std_col_test = {}

for i in range(len(headers)):
    xstd_train, xstd_test, ystd_train, ystd_test = ms.train_test_split(x, y_std, random_state=1)
    std_col_train[headers[i]] = (xstd_train, ystd_train)
    std_col_test[headers[i]] = (xstd_test, ystd_test)

In [141]:
svr, svc = svm.SVR(), svm.SVC()
svr_out, svc_out = {}, {}
svr_exc, svc_exc = [], []

for i in headers:
    xtrain, ytrain = col_train[i]
    xtest, ytest = col_test[i]
    
    try:
        svr.fit(xtrain, ytrain)
        svr_out[i] = svr.score(xtest, ytest)
    except:
        svr_exc.append(i)
    
    try:
        svc.fit(xtrain, ytrain)
        svc_out[i] = svc.score(xtest, ytest)
    except:
        svc_exc.append(i)
    
print("SVR was unable to be run on {}".format(", ".join(svr_exc)))
print("SVC was unable to be run on {}".format(", ".join(svc_exc)))

pd.DataFrame(data={'SVR': svr_out, 'SVC': svc_out}).fillna("N/A").transpose()

NameError: name 'traceback' is not defined

In [107]:
ridge_reg, ridge_clf = lm.Ridge(), lm.RidgeClassifier()
ridge_reg_out, ridge_clf_out = {}, {}
ridge_reg_exc, ridge_clf_exc = [], []

for i in headers:
    xtrain, ytrain = col_train[i]
    xtest, ytest = col_test[i]
    
    try:
        ridge_reg.fit(xtrain, ytrain)
        ridge_reg_out[i] = ridge_reg.score(xtest, ytest)
    except:
        ridge_reg_exc.append(i)
    
    try:
        ridge_clf.fit(xtrain, ytrain)
        ridge_clf_out[i] = ridge_clf.score(xtest, ytest)
        
    except:
        ridge_clf_exc.append(i)
    
print("Ridge Reg was unable to be run on {}".format(", ".join(ridge_reg_exc)))
print("Ridge Clf was unable to be run on {}".format(", ".join(ridge_clf_exc)))

pd.DataFrame(data={'Ridge Reg': ridge_reg_out,
                   'Ridge Clf': ridge_clf_out}).fillna("N/A").transpose()

Ridge Reg was unable to be run on Genotype, Density, Nitrogen, Hormone, PKID
Ridge Clf was unable to be run on Kernelwt, Lipidwt, wtpercent, PALMETIC, LINOLEIC, OLEIC, STEARIC


,Density,Genotype,Hormone,INDICATOR,Kernelwt,LINOLEIC,Lipidwt,Nitrogen,OLEIC,PALMETIC,PKID,STEARIC,wtpercent
Ridge Clf,0.833333,1,0.0833333,1,N/A,N/A,N/A,1,N/A,N/A,1,N/A,N/A
Ridge Reg,N/A,N/A,N/A,1,-0.534945,0.352986,-0.385783,N/A,-0.0650883,-0.0750675,N/A,-0.148973,-0.100502


In [167]:
knn_reg, knn_clf = neigh.KNeighborsRegressor(n_neighbors=3), neigh.KNeighborsClassifier(n_neighbors=3)
knn_reg_out, knn_clf_out = {"Explained V": {},"Mean AE": {}, "Mean SE": {}, "Median AE": {}, "r^2": {}, "Confusion": {}}, {}
knn_reg_exc, knn_clf_exc = [], []

for i in headers:
    xtrain, ytrain = col_train[i]
    xtest, ytest = col_test[i]
    
    if i in num_headers:
        xtrain, ytrain = pre.scale(xtrain), pre.scale(ytrain)
        xtest, ytest = pre.scale(xtest), pre.scale(ytest)
    
    try:
        knn_reg.fit(xtrain, ytrain)
#         knn_reg_out[i] = knn_reg.score(xtest, ytest)
        knn_reg_pred = knn_reg.predict(xtest)
        knn_reg_out["Explained V"][i] = metrics.explained_variance_score(y_true=ytest, y_pred=knn_reg_pred)
        knn_reg_out["Mean AE"][i] = metrics.mean_absolute_error(y_true=ytest, y_pred=knn_reg_pred)
        knn_reg_out["Mean SE"][i] = np.sqrt(metrics.mean_squared_error(y_true=ytest, y_pred=knn_reg_pred))
        knn_reg_out["Median AE"][i] = metrics.median_absolute_error(y_true=ytest, y_pred=knn_reg_pred)
        knn_reg_out["r^2"][i] = metrics.r2_score(y_true=ytest, y_pred=knn_reg_pred)
        knn_reg_out["Confusion"][i] = metrics.confusion_matrix(y_true=ytest, y_pred=knn_reg_pred)
        
    except:
        knn_reg_exc.append(i)
    
    try:
        knn_clf.fit(xtrain, ytrain)
        knn_clf_out[i] = knn_clf.score(xtest, ytest)
        
    except:
        knn_clf_exc.append(i)
    
print("KNN Reg was unable to be run on {}".format(", ".join(knn_reg_exc)))
print("KNN Clf was unable to be run on {}".format(", ".join(knn_clf_exc)))

if len(set(knn_reg_exc).intersection(knn_clf_exc)) >= 1:
    print("Both were unable to be run on {}".format(", ".join(set(knn_reg_exc).intersection(knn_clf_exc))))

knn_df = pd.DataFrame(data={'KNN Explained V': knn_reg_out['Explained V'],
                            'KNN Reg Mean AE': knn_reg_out["Mean AE"],
                            'KNN Reg RMSE': knn_reg_out["Mean SE"],
                            'KNN Reg Median AE': knn_reg_out["Median AE"],
                            'KNN Reg r^2': knn_reg_out["r^2"],
                            'KNN Clf': knn_clf_out}).fillna("N/A").transpose()

KNN Reg was unable to be run on Genotype, Density, Nitrogen, Hormone, PKID
KNN Clf was unable to be run on Kernelwt, Lipidwt, wtpercent, PALMETIC, LINOLEIC, OLEIC, STEARIC


C:\Users\mjs3607\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\utils\validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, _DataConversionWarning)


,Density,Genotype,Hormone,INDICATOR,Kernelwt,LINOLEIC,Lipidwt,Nitrogen,OLEIC,PALMETIC,PKID,STEARIC,wtpercent
KNN Clf,0.75,0.916667,0.166667,1,N/A,N/A,N/A,0.916667,N/A,N/A,1,N/A,N/A
KNN Explained V,N/A,N/A,N/A,1,0.0309509,-0.677144,-0.421347,N/A,-0.301789,-0.251865,N/A,-0.389604,-0.144458
KNN Reg Mean AE,N/A,N/A,N/A,0,0.797134,0.997656,0.953412,N/A,0.877991,0.91286,N/A,0.846108,0.768024
KNN Reg Median AE,N/A,N/A,N/A,0,0.676002,0.633096,0.89876,N/A,0.727611,0.941082,N/A,0.647412,0.696496
KNN Reg RMSE,N/A,N/A,N/A,0,0.984658,1.29507,1.19423,N/A,1.14196,1.12118,N/A,1.17948,1.07286
KNN Reg r^2,N/A,N/A,N/A,1,0.0304477,-0.677207,-0.42619,N/A,-0.304076,-0.257053,N/A,-0.391163,-0.151029


In [ ]:
'''KNN Confusion Matrices'''

for i in knn_reg_out['Confusion']:
    

In [169]:
knn_std_out = {"Explained V": {},"Mean AE": {}, "Mean SE": {}, "Median AE": {}, "r^2": {}}
knn_std_exc = []

for i in headers:
    xtrain, ytrain = std_col_train[i]
    xtest, ytest = std_col_test[i]
    
    if i in num_headers:
        xtrain, ytrain = pre.scale(xtrain), pre.scale(ytrain)
        xtest, ytest = pre.scale(xtest), pre.scale(ytest)
    
    try:
        knn_reg.fit(xtrain, ytrain)
#         knn_reg_out[i] = knn_reg.score(xtest, ytest)
        knn_std_pred = knn_reg.predict(xtest)
        knn_std_out["Explained V"][i] = metrics.explained_variance_score(y_true=ytest, y_pred=knn_std_pred)
        knn_std_out["Mean AE"][i] = metrics.mean_absolute_error(y_true=ytest, y_pred=knn_std_pred)
        knn_std_out["Mean SE"][i] = np.sqrt(metrics.mean_squared_error(y_true=ytest, y_pred=knn_std_pred))
        knn_std_out["Median AE"][i] = metrics.median_absolute_error(y_true=ytest, y_pred=knn_std_pred)
        knn_std_out["r^2"][i] = metrics.r2_score(y_true=ytest, y_pred=knn_std_pred)
        
    except:
        knn_std_exc.append(i)
    
print("KNN Reg was unable to be run on {}".format(", ".join(knn_std_exc)))
pd.DataFrame(data={'KNN Explained V': knn_std_out['Explained V'],
                   'KNN Reg Mean AE': knn_std_out["Mean AE"],
                   'KNN Reg RMSE': knn_std_out["Mean SE"],
                   'KNN Reg Median AE': knn_std_out["Median AE"],
                   'KNN Reg r^2': knn_std_out["r^2"]}).fillna("N/A").transpose()

ValueError: could not convert string to float: 'UCN'

In [120]:
'''Clustering'''
kmeans_out = {}
kmeans_exc = []

for i in headers:
    xtrain, ytrain = col_train[i]
    xtest, ytest = col_test[i]
    
    if i in cat_headers:
        num_cat = len(set(ytrain).union(set((ytest))))
        
        try:
            kmeans = KMeans(n_clusters=num_cat)
            kmeans.fit(xtrain, ytrain)
            kmeans_out[i] = kmeans.score(xtest, ytest)
        except:
            kmeans_exc.append(i)

print("KMeans was unable to be run on {}".format(", ".join(kmeans_exc)))

pd.DataFrame(data={'KMeans': kmeans_out}).fillna("N/A").transpose()

KMeans was unable to be run on Genotype, Density, Nitrogen, Hormone


""
KMeans
